## HPO Phenotype annotation

http://human-phenotype-ontology.github.io/documentation.html
http://compbio.charite.de/jenkins/job/hpo.annotations/lastStableBuild/artifact/misc/phenotype_annotation.tab

I think I've misintepreted what this is... This is a list of patients and their symptoms?

In [1]:
import pandas as pd
import gzip
import json
import os
from collections import defaultdict
from tqdm import tqdm
import pandas as pd

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.mydisease.HPO_pheno

In [5]:
names = ['db', 'db_obj_id', 'db_name', 'qualifier', 'hpo_id', 'db_ref', 'evidence_code', 
         'onset_modifier', 'frequency_modifier', 'with', 'aspect', 'synonym', 'date', 'assigned_by']
df = pd.read_csv("/home/gstupp/projects/biothings/mydisease/mydisease/data/phenotype_annotation.tab", 
            delimiter='\t', names = names)
df.query("db != 'DECIPHER'").head()

,db,db_obj_id,db_name,qualifier,hpo_id,db_ref,evidence_code,onset_modifier,frequency_modifier,with,aspect,synonym,date,assigned_by
285,OMIM,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN,HP:0000028,OMIM:100050,IEA,NaN,NaN,NaN,O,NaN,2009.07.24,HPO:skoehler
286,OMIM,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN,HP:0000049,OMIM:100050,IEA,NaN,NaN,NaN,O,NaN,2009.02.17,HPO
287,OMIM,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN,HP:0000175,OMIM:100050,TAS,NaN,NaN,NaN,O,NaN,2012.07.16,HPO:probinson
288,OMIM,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN,HP:0000202,OMIM:100050,IEA,NaN,NaN,NaN,O,NaN,2010.06.18,HPO:skoehler
289,OMIM,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN,HP:0000204,OMIM:100050,TAS,NaN,NaN,NaN,O,NaN,2009.02.17,HPO:probinson


In [21]:
columns_keep = ['hpo_id', 'evidence_code', 'onset_modifier', 'frequency_modifier', 'with',
       'aspect', 'synonym', 'date', 'assigned_by']
total = len(set(df.db_ref))
d = {}
for diseaseID, subdf in tqdm(df.groupby("db_ref"), total=total):
    sub = subdf[columns_keep].to_dict(orient="records")
    sub = [{k:v for k,v in s.items() if v==v} for s in sub] # get rid of nulls
    db.update_one({'_id':diseaseID}, {'$set':{"phenotypes": sub}}, upsert=True)
    d[diseaseID] = sub

100%|█████████▉| 10091/10092 [00:18<00:00, 534.62it/s]


In [32]:
d['DECIPHER:1']

[{'aspect': 'O',
  'assigned_by': 'HPO:skoehler',
  'date': '2013.05.29',
  'evidence_code': 'IEA',
  'hpo_id': 'HP:0000252'},
 {'aspect': 'O',
  'assigned_by': 'HPO:skoehler',
  'date': '2013.05.29',
  'evidence_code': 'IEA',
  'hpo_id': 'HP:0001249'},
 {'aspect': 'O',
  'assigned_by': 'HPO:skoehler',
  'date': '2013.05.29',
  'evidence_code': 'IEA',
  'hpo_id': 'HP:0001250'},
 {'aspect': 'O',
  'assigned_by': 'HPO:skoehler',
  'date': '2013.05.29',
  'evidence_code': 'IEA',
  'hpo_id': 'HP:0001252'},
 {'aspect': 'O',
  'assigned_by': 'HPO:skoehler',
  'date': '2013.05.29',
  'evidence_code': 'IEA',
  'hpo_id': 'HP:0001518'}]

In [33]:
db.find_one("DECIPHER:1")

{'_id': 'DECIPHER:1',
 'phenotypes': [{'aspect': 'O',
   'assigned_by': 'HPO:skoehler',
   'date': '2013.05.29',
   'evidence_code': 'IEA',
   'hpo_id': 'HP:0000252'},
  {'aspect': 'O',
   'assigned_by': 'HPO:skoehler',
   'date': '2013.05.29',
   'evidence_code': 'IEA',
   'hpo_id': 'HP:0001249'},
  {'aspect': 'O',
   'assigned_by': 'HPO:skoehler',
   'date': '2013.05.29',
   'evidence_code': 'IEA',
   'hpo_id': 'HP:0001250'},
  {'aspect': 'O',
   'assigned_by': 'HPO:skoehler',
   'date': '2013.05.29',
   'evidence_code': 'IEA',
   'hpo_id': 'HP:0001252'},
  {'aspect': 'O',
   'assigned_by': 'HPO:skoehler',
   'date': '2013.05.29',
   'evidence_code': 'IEA',
   'hpo_id': 'HP:0001518'}]}

In [34]:
db_hpo = client.mydisease.HPO
list(db_hpo.find({'_id':{'$in':[x['hpo_id'] for x in db.find_one("DECIPHER:1")['phenotypes']]}},{'name':1}))

[{'_id': 'HP:0000252', 'name': 'Microcephaly'},
 {'_id': 'HP:0001249', 'name': 'Intellectual disability'},
 {'_id': 'HP:0001250', 'name': 'Seizures'},
 {'_id': 'HP:0001252', 'name': 'Muscular hypotonia'},
 {'_id': 'HP:0001518', 'name': 'Small for gestational age'}]